In [1]:
# ✅ UNI-2h : téléchargement & chargement
from huggingface_hub import hf_hub_download
import timm, torch
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
print(torch.cuda.is_available(), torch.cuda.get_device_name(0))


True NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
from huggingface_hub import login
import os

HF_TOKEN = os.getenv("HF_TOKEN")
# Vérifie qu'il est chargé (ne jamais l'afficher publiquement)
assert HF_TOKEN is not None, "HF_TOKEN n'est pas défini dans l'environnement !"

In [4]:
# Vérifie l’accès (doit télécharger un petit fichier)
cfg_path = hf_hub_download("MahmoodLab/UNI2-h", "config.json")
print("UNI2-h config:", cfg_path)

UNI2-h config: /workspace/.cache/huggingface/hub/models--MahmoodLab--UNI2-h/snapshots/d517a8dd47902dd7c308b3c36f63bce47e7b9a43/config.json


In [5]:
from huggingface_hub import hf_hub_download
p = hf_hub_download("MahmoodLab/UNI2-h", "config.json")
print("OK, accès UNI-2h →", p)


OK, accès UNI-2h → /workspace/.cache/huggingface/hub/models--MahmoodLab--UNI2-h/snapshots/d517a8dd47902dd7c308b3c36f63bce47e7b9a43/config.json


In [6]:
import torch
# Vidage RAM GPU
torch.cuda.empty_cache()

In [7]:
# ✅ Load UNI-2h with the right architecture args
import torch, timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

device = "cuda" if torch.cuda.is_available() else "cpu"

timm_kwargs = {
    "img_size": 224,
    "patch_size": 14,
    "depth": 24,
    "num_heads": 24,
    "embed_dim": 1536,
    "mlp_ratio": 2.66667 * 2,   # as in UNI-2h
    "init_values": 1e-5,
    "num_classes": 0,
    "no_embed_class": True,
    "reg_tokens": 8,
    "dynamic_img_size": True,
    "mlp_layer": timm.layers.SwiGLUPacked,
    "act_layer": torch.nn.SiLU,
}

uni = timm.create_model("hf-hub:MahmoodLab/UNI2-h", pretrained=True, **timm_kwargs).eval().to(device)

# Preprocess transform from the model’s config
cfg = resolve_data_config(uni.pretrained_cfg, model=uni)
tfm = create_transform(**cfg)

print("UNI-2h ready →", next(uni.parameters()).dtype, "| device:", device)


UNI-2h ready → torch.float32 | device: cuda


In [8]:
import torch
x = torch.randn(1, 3, 224, 224, device=device)
with torch.inference_mode():
    y = uni(x)   # should be shape (1, 1536)
print("Embedding shape:", tuple(y.shape))


Embedding shape: (1, 1536)


In [ ]:
# Contexte minimal requis
device = "cuda" if torch.cuda.is_available() else "cpu"
assert 'uni' in globals() and 'tfm' in globals(), "Charge d'abord UNI-2h (timm) et create_transform(tfm)."

# Workaround pour transformers.modeling_layers manquant
# Dans transformers 4.45.2, modeling_layers n'existe pas.
# diffusers importe peft (même sans LoRA), et peft essaie d'importer
# transformers.modeling_layers.GradientCheckpointingLayer lors de son initialisation.
# On crée un module factice avant l'import de diffusers pour satisfaire peft.
import sys
import types
import transformers

if 'transformers.modeling_layers' not in sys.modules:
    # Classe factice (on n'utilise pas LoRA, donc pas besoin d'implémentation réelle)
    class GradientCheckpointingLayer:
        """Classe factice pour compatibilité avec peft"""
        pass
    
    # Créer et enregistrer le module factice
    modeling_layers_module = types.ModuleType('transformers.modeling_layers')
    modeling_layers_module.GradientCheckpointingLayer = GradientCheckpointingLayer
    sys.modules['transformers.modeling_layers'] = modeling_layers_module
    transformers.modeling_layers = modeling_layers_module

from diffusers import DiffusionPipeline, AutoencoderKL
from pathlib import Path
from PIL import Image
import torch
import os

# --- Emplacement du projet quand on lance depuis notebooks/ ---
PROJECT_ROOT  = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
DATA_ROOT     = Path(os.getenv("DATA_ROOT", "/workspace/data")).resolve()

# Dossiers globaux du projet
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
MODELS_DIR    = PROJECT_ROOT / "models"
SAMPLES_DIR   = PROJECT_ROOT / "samples" / "07_diffusion_model"
OUTPUTS_DIR   = PROJECT_ROOT / "outputs" / "07_diffusion_model"
CHECKPOINTS_DIR = PROJECT_ROOT / "checkpoints" / "07_diffusion_model"

for d in [ARTIFACTS_DIR, MODELS_DIR, SAMPLES_DIR, OUTPUTS_DIR, CHECKPOINTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

OUT_DIR = OUTPUTS_DIR / "pixcell_out_256"
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [10]:
# === Toggles mémoire : checkpointing, attn slicing/xformers, VAE tiling ===
try:
    BACKBONE.enable_gradient_checkpointing()
    print("✓ Gradient checkpointing activé")
except Exception as e:
    print("↷ Checkpointing indisponible sur ce backbone :", type(e).__name__)

# xFormers / FlashAttention / slicing
ok_attn = False
try:
    BACKBONE.enable_xformers_memory_efficient_attention()
    print("✓ xFormers attention"); ok_attn = True
except Exception:
    try:
        from diffusers.models.attention_processor import AttnProcessor2_0
        BACKBONE.set_attn_processor(AttnProcessor2_0())
        print("✓ AttnProcessor2_0"); ok_attn = True
    except Exception:
        try:
            BACKBONE.enable_attention_slicing("max")
            print("✓ Attention slicing (fallback)"); ok_attn = True
        except Exception as e:
            print("↷ Pas de backend attn optimisé :", type(e).__name__)

# VAE encode/decode par tuiles (gros gain mémoire)
try:
    VAE.enable_tiling(); VAE.enable_slicing()
    print("✓ VAE tiling + slicing")
except Exception as e:
    print("↷ VAE tiling/slicing non supporté :", type(e).__name__)

# Option : réduis la résolution si tu es short
IMG_SIZE = int(globals().get('IMG_SIZE', 256))
# IMG_SIZE = 192  # ← décommente si besoin d’un cran de marge


↷ Checkpointing indisponible sur ce backbone : NameError
↷ Pas de backend attn optimisé : NameError
↷ VAE tiling/slicing non supporté : NameError


In [11]:
# %% 🧹 OOM cleanup — ferme figures, libère tenseurs, retire hooks, vide caches CUDA
import gc, sys
import torch
import matplotlib
import types

def _mb(x): 
    return f"{x/1024/1024:.1f} MB"

# 1) Fermer toutes les figures Matplotlib
try:
    import matplotlib.pyplot as plt
    plt.close('all')
    print("✔️ Figures Matplotlib fermées.")
except Exception as e:
    print("ℹ️ plt.close('all') ignoré:", e)

# 2) Retirer le hook 'noise-only' si présent
try:
    _pipe = globals().get("pipe", None) or globals().get("pipe256", None)
    if _pipe is not None and getattr(_pipe.transformer, "_noise_only_handle", None) is not None:
        _pipe.transformer._noise_only_handle.remove()
        _pipe.transformer._noise_only_handle = None
        print("✔️ Hook 'noise-only' retiré.")
except Exception as e:
    print("ℹ️ Retrait hook ignoré:", e)

# 3) Supprimer variables temporaires fréquentes
to_del = [
    "out","images","pils","pils_show","x01","latents","noise","noisy","pred","target",
    "pos_uni","neg_uni","pos_enc","neg_enc","g","batch","dl_iter"
]
freed = 0
for name in to_del:
    if name in globals():
        try:
            obj = globals()[name]
            del globals()[name]
            freed += 1
        except Exception:
            pass
print(f"✔️ Variables temporaires supprimées: {freed}")

# 4) Forcer le GC Python
collected = gc.collect()
print(f"✔️ GC collecté: {collected} objets")

# 5) Vidage des caches CUDA
if torch.cuda.is_available():
    try:
        torch.cuda.synchronize()
    except Exception:
        pass
    try:
        before_alloc = torch.cuda.memory_allocated()
        before_resrv = torch.cuda.memory_reserved()
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        torch.cuda.reset_peak_memory_stats()
        after_alloc  = torch.cuda.memory_allocated()
        after_resrv  = torch.cuda.memory_reserved()
        print(f"✔️ CUDA cache vidé. Alloc {_mb(before_alloc)} → {_mb(after_alloc)} | Reserved {_mb(before_resrv)} → {_mb(after_resrv)}")
    except Exception as e:
        print("ℹ️ Cleanup CUDA ignoré:", e)
else:
    print("ℹ️ CUDA non disponible.")

print("✅ Ménage fini — si besoin, relance la génération avec un batch/steps plus petits (ex: N_SHOW=2, GEN_STEPS=12).")


✔️ Figures Matplotlib fermées.
✔️ Variables temporaires supprimées: 0
✔️ GC collecté: 20 objets
✔️ CUDA cache vidé. Alloc 2634.5 MB → 2634.5 MB | Reserved 2684.0 MB → 2664.0 MB
✅ Ménage fini — si besoin, relance la génération avec un batch/steps plus petits (ex: N_SHOW=2, GEN_STEPS=12).


In [12]:
# Filtre des warnings liés aux attributs de config non attendus
# (ces warnings sont inoffensifs - les attributs sont simplement ignorés)
import warnings
warnings.filterwarnings("ignore", message=".*config attributes.*were passed.*but are not expected.*")
warnings.filterwarnings("ignore", message=".*Keyword arguments.*are not expected.*and will be ignored.*")

# VAE SD3.5 (nécessite accès HF)
vae = AutoencoderKL.from_pretrained(
    "stabilityai/stable-diffusion-3.5-large",
    subfolder="vae",
    torch_dtype=torch.float16,
)

# PixCell-256
pipe256 = DiffusionPipeline.from_pretrained(
    "StonyBrook-CVLab/PixCell-256",
    vae=vae,
    custom_pipeline="StonyBrook-CVLab/PixCell-pipeline",
    trust_remote_code=True,
    torch_dtype=torch.float16,
).to(device)

# Optim VRAM
pipe256.enable_attention_slicing()
pipe256.vae.enable_slicing()
# pipe256.vae.enable_tiling()  # optionnel

# --- UNI → embed ---
@torch.inference_mode()
def uni_embed_from_image(img: Image.Image, device=device):
    x = tfm(img.convert("RGB")).unsqueeze(0).to(device)
    with torch.autocast(device_type="cuda", enabled=(device=="cuda")):
        e = uni(x)               # (1, 1536)
    return e.unsqueeze(1)         # (1, 1, 1536)

# --- PixCell-256 génération sûre ---
@torch.inference_mode()
def generate_pixcell256(ref_img: Image.Image,
                        guidance_scale: float = 2.0,
                        n: int = 1,                 # laisser 1 si VRAM limite
                        seed: int | None = 1234,
                        steps: int = 20):
    # NOTE: si 'device' est déjà global, NE le redéclare pas ici.
    g = torch.Generator(device=device).manual_seed(seed) if seed is not None else None

    emb = uni_embed_from_image(ref_img, device=device)       # (B=1,1,1536) float32
    # (facultatif) cast en fp16 pour économiser la VRAM :
    emb = emb.to(device, dtype=torch.float16)
    B = emb.shape[0]

    uncond = pipe256.get_unconditional_embedding(B).to(device=emb.device, dtype=emb.dtype)

    # ⚠️ TENSEURS (B,), float32, sur le bon device — pour cond ET uncond
    added_cond = {
        "resolution":   torch.full((B,), 256.0, device=device, dtype=torch.float32),
        "aspect_ratio": torch.full((B,),   1.0, device=device, dtype=torch.float32),
    }
    negative_added_cond = {
        "resolution":   torch.full((B,), 256.0, device=device, dtype=torch.float32),
        "aspect_ratio": torch.full((B,),   1.0, device=device, dtype=torch.float32),
    }

    with torch.autocast(device_type="cuda", enabled=(device=="cuda")):
        result = pipe256(
            uni_embeds=emb,
            negative_uni_embeds=uncond,
            guidance_scale=guidance_scale,     # ≈ 2.0 pour 256
            num_inference_steps=steps,
            num_images_per_prompt=n,
            generator=g,
            height=256,                        # <-- clé !
            width=256,                         # <-- clé !
            # added_cond_kwargs=added_cond,
            # negative_added_cond_kwargs=negative_added_cond,
        )

    return result.images

Keyword arguments {'trust_remote_code': True} are not expected by PixCellPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

The config attributes {'double_self_attention': False, 'num_vector_embeds': None, 'only_cross_attention': False, 'use_linear_projection': False} were passed to PixCellTransformer2DModel, but are not expected and will be ignored. Please verify your config.json configuration file.
The config attributes {'flow_shift': 1.0, 'use_flow_sigmas': False} were passed to DPMSolverMultistepScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


In [13]:
# ---- 1) Préparer les entrées (expose emb/kwargs pour debug) ----
@torch.inference_mode()
def prepare_pixcell256_inputs(ref_img: Image.Image, device=device):
    x = tfm(ref_img.convert("RGB")).unsqueeze(0).to(device)
    with torch.autocast(device_type="cuda", enabled=(device=="cuda")):
        e = uni(x)
    emb = e.unsqueeze(1).to(device, dtype=torch.float16)   # (1,1,1536)

    B = emb.shape[0]
    uncond = pipe256.get_unconditional_embedding(B)

    # requis par PixCell -> transformer -> AdaLN -> emb.forward
    added_cond = {
      "resolution":   torch.full((B,), 256.0, device=device, dtype=torch.float32),
      "aspect_ratio": torch.full((B,),   1.0, device=device, dtype=torch.float32),
    }
    negative_added_cond = {
      "resolution":   torch.full((B,), 256.0, device=device, dtype=torch.float32),
      "aspect_ratio": torch.full((B,),   1.0, device=device, dtype=torch.float32),
    }
    return emb, uncond, added_cond, negative_added_cond

# ---- 2) Appeler la pipeline avec ces entrées ----
@torch.inference_mode()
def run_pixcell256(emb, uncond,
                   guidance_scale: float = 2.0, steps: int = 20, n: int = 1, seed: int | None = 2025):
    g = torch.Generator(device=emb.device).manual_seed(seed) if seed is not None else None

    if uncond.dtype != emb.dtype or uncond.device != emb.device:
        uncond = uncond.to(device=emb.device, dtype=emb.dtype)

    B = emb.shape[0]
    # Ces deux tenseurs sont OBLIGATOIRES pour PixArtAlphaCombinedTimestepSizeEmbeddings
    resolution   = torch.full((B,), 256.0, device=emb.device, dtype=torch.float32)
    aspect_ratio = torch.full((B,),   1.0, device=emb.device, dtype=torch.float32)

    added_cond = {"resolution": resolution, "aspect_ratio": aspect_ratio}

    with torch.autocast(device_type="cuda", enabled=(emb.device.type=="cuda")):
        result = pipe256(
            uni_embeds=emb,
            negative_uni_embeds=uncond,
            guidance_scale=guidance_scale,
            num_inference_steps=steps,
            num_images_per_prompt=n,
            generator=g,
            height=256,
            width=256,
            added_cond_kwargs=added_cond,  # ← la clé essentielle
        )

    return result.images

In [14]:
# --- Monkey-patch robuste de PixArtAlphaCombinedTimestepSizeEmbeddings.forward ---
import types, torch

adaln   = pipe256.transformer.adaln_single          # AdaLayerNormSingle
emb_mod = adaln.emb                                 # PixArtAlphaCombinedTimestepSizeEmbeddings
orig_forward = emb_mod.forward

def _patched_forward(self, *f_args, **f_kwargs):
    """
    Compatibilité maximale avec tous les schémas d'appel:
    forward(self, timestep, resolution, aspect_ratio, [batch_size], [hidden_dtype])
    + variantes avec kwargs.
    Injecte resolution/aspect_ratio si absents.
    """
    # --- 1) Extraire les params quelle que soit la forme d'appel
    # Positional
    timestep = f_args[0] if len(f_args) > 0 else f_kwargs.get("timestep", None)

    # resolution / aspect_ratio : prioriser kwargs, sinon args[1], args[2]
    resolution   = f_kwargs.get("resolution",   None)
    aspect_ratio = f_kwargs.get("aspect_ratio", None)
    if resolution is None and len(f_args) > 1:
        resolution = f_args[1]
    if aspect_ratio is None and len(f_args) > 2:
        aspect_ratio = f_args[2]

    # batch_size / hidden_dtype : gérer positionnels [3],[4] ou kwargs
    batch_size   = f_kwargs.get("batch_size",   None)
    hidden_dtype = f_kwargs.get("hidden_dtype", None)
    if batch_size is None and len(f_args) > 3:
        batch_size = f_args[3]
    if hidden_dtype is None and len(f_args) > 4:
        hidden_dtype = f_args[4]

    # --- 2) Déterminer B et device sans supposer le type exact de timestep
    # B
    if batch_size is not None:
        if isinstance(batch_size, torch.Tensor):
            B = int(batch_size.item())
        else:
            try:
                B = int(batch_size)
            except Exception:
                B = 1
    else:
        try:
            B = int(getattr(timestep, "shape", [1])[0])
        except Exception:
            B = 1

    # device
    if isinstance(timestep, torch.Tensor):
        device = timestep.device
    else:
        # fallback: tenter de récupérer un device depuis un param interne… sinon CUDA si dispo
        try:
            device = next(self.parameters()).device
        except Exception:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- 3) Injecter les valeurs manquantes
    if resolution is None:
        resolution = torch.full((B,), 256.0, device=device, dtype=torch.float32)
    if aspect_ratio is None:
        aspect_ratio = torch.full((B,),  1.0, device=device, dtype=torch.float32)

    # --- 4) Appeler l'original proprement (éviter les conflits positionnels)
    return orig_forward(
        timestep,
        resolution,
        aspect_ratio,
        batch_size=batch_size,
        hidden_dtype=hidden_dtype,
    )

# Appliquer le patch
emb_mod.forward = types.MethodType(_patched_forward, emb_mod)
print("✅ Patched PixArtAlphaCombinedTimestepSizeEmbeddings.forward (signature-compatible)")


✅ Patched PixArtAlphaCombinedTimestepSizeEmbeddings.forward (signature-compatible)


In [15]:
# Helper simple si le TIFF n'est pas exactement 256x256
from PIL import Image
def load_ref_image_any(path, target_hw=(256, 256)):
    img = Image.open(path)
    if getattr(img, "n_frames", 1) > 1:
        img.seek(0)
    if img.mode not in ("RGB",):
        img = img.convert("RGB")
    if target_hw and img.size != (target_hw[1], target_hw[0]):
        img = img.resize((target_hw[1], target_hw[0]), Image.BICUBIC)
    return img

In [16]:
# Charge/resize le TIFF si besoin
# Utiliser DATA_ROOT défini dans la cellule précédente
if "DATA_ROOT" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    DATA_ROOT = Path(os.getenv("DATA_ROOT", "/workspace/data")).resolve()
img = load_ref_image_any(str(DATA_ROOT / "NCT-CRC-HE-100K/TUM/TUM-AAHFEMQY.tif"), target_hw=(256,256))

# 1) Prépare les tenseurs
emb, uncond, added_cond, negative_added_cond = prepare_pixcell256_inputs(img, device=device)

# 2) Vérifie shapes/devices/dtypes (tu peux copier-coller ces prints)
print("emb:", emb.shape, emb.device, emb.dtype)
print("uncond:", uncond.shape, uncond.device, uncond.dtype)

emb: torch.Size([1, 1, 1536]) cuda:0 torch.float16
uncond: torch.Size([1, 1, 1536]) cuda:0 torch.float16


In [17]:
# 3) Génère une image

outs = run_pixcell256(emb, uncond, guidance_scale=2.0, steps=20, n=1, seed=2025)

# 4) Sauvegarde
OUT_DIR.mkdir(parents=True, exist_ok=True)
outs[0].save(OUT_DIR / "pixcell256_ok.png")
print("✅ écrit →", OUT_DIR / "pixcell256_ok.png")

  0%|          | 0/20 [00:00<?, ?it/s]

✅ écrit → /workspace/outputs/07_diffusion_model/pixcell_out_256/pixcell256_ok.png


In [18]:
# Affichage des repo utilisés
import json, re
from pathlib import Path

candidates = []
paths_to_try = [
    Path("/notebook/07_Diffusion_model.ipynb"),
    Path.cwd() / "07_Diffusion_model.ipynb",
] + list(Path("/").rglob("07_Diffusion_model.ipynb"))  # large, mais utile si besoin

for p in paths_to_try:
    if p.exists():
        try:
            nb = json.loads(p.read_text())
            for cell in nb.get("cells", []):
                if cell.get("cell_type") == "code":
                    src = "".join(cell.get("source", []))
                    candidates += re.findall(r'from_pretrained\s*\(\s*[\'"]([^\'"]+)[\'"]', src)
        except Exception:
            pass

candidates = sorted(set(candidates))
print("Repos trouvés :", candidates)


Repos trouvés : ['StonyBrook-CVLab/PixCell-256', 'stabilityai/stable-diffusion-3.5-large']


## Généralisation - Génération d'images depuis un dossier de modèles

In [19]:
from pathlib import Path
from PIL import Image
import torch, math, random, time

# =========================
# ⚙️ CONFIG GÉNÉRATION
# =========================
# Utiliser DATA_ROOT défini dans la cellule précédente
if "DATA_ROOT" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    DATA_ROOT = Path(os.getenv("DATA_ROOT", "/workspace/data")).resolve()
REF_DIR = DATA_ROOT / "NCT-CRC-HE-100K/TUM"

TOTAL_IMAGES     = 20          # ← combien d’images au total
MAX_REF_IMAGES   = 8           # ← nb max d’images de référence différentes (ajuste si tu veux)
GUIDANCE_SCALE   = 2.0         # passer jusqu'à 2.3 pour diversité augmentée
STEPS            = 20          # augmenter pour plus de diversité 
BASE_SEED        = 2025        # ← pour reproductibilité
SHUFFLE_REFS     = True        # ← mélange les références

# Extensions d’images acceptées
EXTS = {".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp", ".webp"}

# =========================
# 🔎 LISTE DES RÉFÉRENCES
# =========================
ref_paths = [p for p in REF_DIR.rglob("*") if p.suffix.lower() in EXTS]
assert len(ref_paths) > 0, f"Aucune image trouvée dans {REF_DIR}"

if SHUFFLE_REFS:
    random.Random(BASE_SEED).shuffle(ref_paths)

# On limite le nombre d'images de référence si besoin
ref_paths = ref_paths[:MAX_REF_IMAGES]
num_refs = len(ref_paths)

# Combien de variantes par référence pour atteindre TOTAL_IMAGES
k_per_ref = max(1, math.ceil(TOTAL_IMAGES / num_refs))

print(f"🔧 {num_refs} image(s) de référence sélectionnées → {k_per_ref} variante(s) / ref ≈ {num_refs*k_per_ref} sorties")
print("Références:", [p.name for p in ref_paths])

# =========================
# 🧪 UTIL : charge + emb UNI
# =========================
@torch.inference_mode()
def emb_from_path(img_path: Path, device="cuda"):
    img = Image.open(img_path).convert("RGB")
    # réutilise ta fonction si elle existe déjà (sinon keep ce bloc)
    x = tfm(img).unsqueeze(0).to(device)
    with torch.autocast(device_type="cuda", enabled=(device=="cuda")):
        e = uni(x)               # (1, 1536)
    emb = e.unsqueeze(1).to(device, dtype=torch.float16)  # (1,1,1536) fp16
    return img, emb

# =========================
# 🚀 GÉNÉRATION
# =========================
generated = []
t0 = time.time()
torch.cuda.empty_cache()

idx_global = 0
for r_idx, ref_path in enumerate(ref_paths):
    try:
        ref_img, emb = emb_from_path(ref_path, device="cuda")
        B = emb.shape[0]
        uncond = pipe256.get_unconditional_embedding(B).to(device=emb.device, dtype=emb.dtype)

        for j in range(k_per_ref):
            if idx_global >= TOTAL_IMAGES:
                break

            seed = BASE_SEED + idx_global
            outs = run_pixcell256(
                emb, uncond,
                guidance_scale=GUIDANCE_SCALE,
                steps=STEPS,
                n=1,
                seed=seed
            )

            # Sauvegarde
            out_img = outs[0]
            out_name = f"TUM_ref{r_idx:02d}_{ref_path.stem[:20]}_s{seed}_gs{GUIDANCE_SCALE}_st{STEPS}.png"
            out_path = OUT_DIR / out_name
            out_img.save(out_path)
            generated.append(out_path)
            idx_global += 1

            # Hygiène VRAM
            if idx_global % 4 == 0:
                torch.cuda.empty_cache()

    except Exception as e:
        print(f"⚠️ Réf #{r_idx} ({ref_path.name}) : skipped → {e}")
        continue

dt = time.time() - t0
print(f"\n✅ Terminé : {len(generated)} images générées en {dt:.1f}s → {OUT_DIR}")
for p in generated:
    print(" •", p.name)


🔧 8 image(s) de référence sélectionnées → 3 variante(s) / ref ≈ 24 sorties
Références: ['TUM-VWHSHCWP.tif', 'TUM-EANRWTLE.tif', 'TUM-KRQNFDYS.tif', 'TUM-AWTSGMLD.tif', 'TUM-GYKNYVCG.tif', 'TUM-TFIEFMEI.tif', 'TUM-LVIYRCGQ.tif', 'TUM-KENYIKEC.tif']


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]


✅ Terminé : 20 images générées en 14.0s → /workspace/outputs/07_diffusion_model/pixcell_out_256
 • TUM_ref00_TUM-VWHSHCWP_s2025_gs2.0_st20.png
 • TUM_ref00_TUM-VWHSHCWP_s2026_gs2.0_st20.png
 • TUM_ref00_TUM-VWHSHCWP_s2027_gs2.0_st20.png
 • TUM_ref01_TUM-EANRWTLE_s2028_gs2.0_st20.png
 • TUM_ref01_TUM-EANRWTLE_s2029_gs2.0_st20.png
 • TUM_ref01_TUM-EANRWTLE_s2030_gs2.0_st20.png
 • TUM_ref02_TUM-KRQNFDYS_s2031_gs2.0_st20.png
 • TUM_ref02_TUM-KRQNFDYS_s2032_gs2.0_st20.png
 • TUM_ref02_TUM-KRQNFDYS_s2033_gs2.0_st20.png
 • TUM_ref03_TUM-AWTSGMLD_s2034_gs2.0_st20.png
 • TUM_ref03_TUM-AWTSGMLD_s2035_gs2.0_st20.png
 • TUM_ref03_TUM-AWTSGMLD_s2036_gs2.0_st20.png
 • TUM_ref04_TUM-GYKNYVCG_s2037_gs2.0_st20.png
 • TUM_ref04_TUM-GYKNYVCG_s2038_gs2.0_st20.png
 • TUM_ref04_TUM-GYKNYVCG_s2039_gs2.0_st20.png
 • TUM_ref05_TUM-TFIEFMEI_s2040_gs2.0_st20.png
 • TUM_ref05_TUM-TFIEFMEI_s2041_gs2.0_st20.png
 • TUM_ref05_TUM-TFIEFMEI_s2042_gs2.0_st20.png
 • TUM_ref06_TUM-LVIYRCGQ_s2043_gs2.0_st20.png
 • TUM_ref

## Alimentation par le DataGenerator

In [20]:
# =========================================================
# 🔧 Import HistoDataset depuis le projet local
# =========================================================
import sys
from pathlib import Path

# Utiliser PROJECT_ROOT défini dans la cellule précédente
if "PROJECT_ROOT" not in globals():
    PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
P9DG_DIR = PROJECT_ROOT / "p9dg"  # Dossier du datagenerator et utilitaires de mapping

# Vérifie et ajoute au PYTHONPATH
if str(P9DG_DIR) not in sys.path:
    sys.path.append(str(P9DG_DIR))

# Import du dataset
from histo_dataset import HistoDataset

print(f"✅ Module histo_dataset importé depuis : {P9DG_DIR}")


✅ Module histo_dataset importé depuis : /workspace/p9dg


In [21]:
# =========================================================
# UNI ↔ PixCell ↔ HistoDataset  (génération multi-classe finale + VAL cross-dataset)
# =========================================================
from torch.utils.data import DataLoader
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm
import torch, time, random, csv, os
from utils.class_mappings import make_idx_mappings

# ----------------------------
# PARAMÈTRES DE GÉNÉRATION
# ----------------------------
# (Re)création propre du dossier de sortie
import shutil

# Utiliser OUTPUTS_DIR défini dans la cellule précédente
if "OUTPUTS_DIR" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    OUTPUTS_DIR = PROJECT_ROOT / "outputs" / "07_diffusion_model"

OUT_DIR = OUTPUTS_DIR / "pixcell_out_histo"
if OUT_DIR.exists():
    assert OUT_DIR.parent == OUTPUTS_DIR
    shutil.rmtree(OUT_DIR)  # ⚠️ supprime tout le contenu s'il existe déjà
OUT_DIR.mkdir(parents=True, exist_ok=True)


GUIDANCE_SCALE = 2.0
STEPS = 28
TOTAL_SAMPLES = 4500            # nombre cible TRAIN (sera réparti par classe)
BATCH_SIZE = 4
N_PER_REF = 1
BASE_SEED = 2025
DEVICE = "cuda"

# --- Activer une mini-boucle sur le split "val" (CRC-VAL-HE-7K) en plus du "train" (NCT-CRC-HE-100K)
VAL_ENABLE = True
VAL_SAMPLES_PER_CLASS = None   # None → calcule automatiquement à 25% de SAMPLES_PER_CLASS
VAL_FRACTION = 0.25

# --- CLASSES À GÉNÉRER ---
# None → intersection auto des classes présentes dans TRAIN et VAL
# ["TUM"] → monoclasse, etc. (sera validé contre les deux datasets)
SELECTED_CLASSES = None

torch.manual_seed(BASE_SEED)
random.seed(BASE_SEED)

# ----------------------------
# 📁 Racines de datasets
# ----------------------------
# Utiliser DATA_ROOT défini dans la cellule précédente
if "DATA_ROOT" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    DATA_ROOT = Path(os.getenv("DATA_ROOT", "/workspace/data")).resolve()
else:
    DATA_ROOT = Path(DATA_ROOT)  # S'assurer que c'est un Path
TRAIN_DATASET_NAME = "NCT-CRC-HE-100K"
VAL_DATASET_NAME   = "CRC-VAL-HE-7K"

TRAIN_ROOT = DATA_ROOT / TRAIN_DATASET_NAME
VAL_ROOT   = DATA_ROOT / VAL_DATASET_NAME

assert TRAIN_ROOT.exists(), f"Chemin introuvable : {TRAIN_ROOT}"
if VAL_ENABLE:
    assert VAL_ROOT.exists(), f"Chemin introuvable : {VAL_ROOT}"

def _list_classes(d: Path):
    return sorted([x.name for x in d.iterdir() if x.is_dir() and not x.name.startswith(".")])

# → 1️⃣ Détection/validation des classes
train_classes = _list_classes(TRAIN_ROOT)
val_classes   = _list_classes(VAL_ROOT) if VAL_ENABLE else train_classes

if SELECTED_CLASSES is None:
    SELECTED_CLASSES = sorted(set(train_classes) & set(val_classes))
    assert len(SELECTED_CLASSES) > 0, "Aucune classe commune entre TRAIN et VAL."
else:
    req = set(SELECTED_CLASSES)
    valid = req & set(train_classes) & set(val_classes)
    missing = req - valid
    SELECTED_CLASSES = sorted(valid)
    assert len(SELECTED_CLASSES) > 0, f"Aucune classe valide parmi la sélection fournie. Manquantes: {sorted(missing)}"
    if missing:
        print(f"[⚠️] Classes ignorées (absentes train/val): {sorted(missing)}")

# → 2️⃣ Calcul automatique du nombre d’images par classe (TRAIN)
N_CLASSES = len(SELECTED_CLASSES)
SAMPLES_PER_CLASS = max(1, TOTAL_SAMPLES // N_CLASSES)
if VAL_SAMPLES_PER_CLASS is None:
    VAL_SAMPLES_PER_CLASS = max(1, int(SAMPLES_PER_CLASS * VAL_FRACTION))

print(f"📂 TRAIN dataset: {TRAIN_DATASET_NAME} | VAL dataset: {VAL_DATASET_NAME if VAL_ENABLE else '—'}")
print(f"⚖️ {N_CLASSES} classes: {SELECTED_CLASSES}")
print(f"📦 TRAIN: {SAMPLES_PER_CLASS} / classe (≈ {SAMPLES_PER_CLASS * N_CLASSES} au total)")
if VAL_ENABLE:
    print(f"📦  VAL : {VAL_SAMPLES_PER_CLASS} / classe (en plus du TRAIN)")

# ----------------------------
# Utilitaires
# ----------------------------
def get_uni_batch_embeddings(pil_batch, device="cuda"):
    """Retourne les embeddings UNI au format (B, N, D), quel que soit le format d'entrée."""
    emb_list = []
    for im in pil_batch:
        e = uni_embed_from_image(im, device=device)
        if e.dim() == 1:
            e = e.unsqueeze(0).unsqueeze(0)
        elif e.dim() == 2:
            e = e.unsqueeze(0)
        emb_list.append(e)
    emb = torch.cat(emb_list, dim=0)
    return emb

def build_loader(split:str, samples_per_class:int):
    # HistoDataset sait router: train→NCT-CRC-HE-100K, val/test→CRC-VAL-HE-7K (split_policy="by_dataset")
    ds = HistoDataset(
        root_data=str(DATA_ROOT),
        split=split,                       # "train" ou "val"
        output_size=256,
        pixel_range="0_1",
        classes=SELECTED_CLASSES,
        balance_per_class=True,
        samples_per_class_per_epoch=samples_per_class,
        vahadane_enable=False,             # rester proche du réel
        return_labels=True,
        apply_quality_filter=False         # comparaison avec réel
    )
    dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
    return ds, dl

def generate_for_split(split:str, samples_per_class:int, start_seed:int, start_idx:int,
                       metadata_rows:list, generated_paths:list):
    ds, dl = build_loader(split, samples_per_class)
    idx_to_name, idx_to_color, _ = make_idx_mappings(ds.class_to_idx)

    print(f"✅ Dataset {split} prêt : {len(ds)} échantillons ({samples_per_class}/classe)")
    pbar = tqdm(dl, desc=f"Génération PixCell ({split})", total=len(dl), leave=False)

    global_idx = start_idx
    base_seed = start_seed

    for batch_idx, (x, labels, paths) in enumerate(pbar):
        pil_batch = [Image.open(p).convert("RGB") for p in paths]
        emb = get_uni_batch_embeddings(pil_batch, device=DEVICE)
        uncond = pipe256.get_unconditional_embedding(emb.shape[0]).to(emb.device, dtype=emb.dtype)

        seeds = [base_seed + global_idx + k for k in range(emb.shape[0])]
        outs_imgs = []
        for k in range(emb.shape[0]):
            outs_k = run_pixcell256(
                emb[k:k+1], uncond[k:k+1],
                guidance_scale=GUIDANCE_SCALE,
                steps=STEPS,
                n=1,
                seed=seeds[k]
            )
            outs_imgs.append(outs_k[0])

        for r, ref_path in enumerate(paths):
            label_idx = int(labels[r])
            label_code = list(ds.class_to_idx.keys())[label_idx]
            label_name = idx_to_name[label_idx]

            class_dir = OUT_DIR / label_code
            class_dir.mkdir(parents=True, exist_ok=True)

            out_img = outs_imgs[r]
            seed = seeds[r]
            ref_name = os.path.splitext(os.path.basename(ref_path))[0]
            out_name = f"{ref_name}__{split}__pixcell_seed{seed:06d}.png"
            out_path = class_dir / out_name
            out_img.save(out_path)
            generated_paths.append(str(out_path))

            metadata_rows.append({
                "global_index": global_idx,
                "split": split,                        # ← clé pour ton split CNN en aval
                "ref_path": str(ref_path),
                "ref_label": label_code,
                "ref_label_full": label_name,
                "ref_color": idx_to_color[label_idx],
                "seed": seed,
                "out_path": str(out_path),
                "guidance_scale": GUIDANCE_SCALE,
                "steps": STEPS
            })
            global_idx += 1

        if (batch_idx + 1) % 4 == 0:
            torch.cuda.empty_cache()

    return global_idx  # prochain index global

# ----------------------------
# BOUCLES DE GÉNÉRATION (TRAIN puis VAL optionnel)
# ----------------------------
torch.cuda.empty_cache()
generated_paths, metadata_rows = [], []
t0 = time.time()
global_idx = 0

# 1) TRAIN (NCT-CRC-HE-100K)
global_idx = generate_for_split("train", SAMPLES_PER_CLASS, BASE_SEED, global_idx,
                                metadata_rows, generated_paths)

# 2) VAL (CRC-VAL-HE-7K)
if VAL_ENABLE:
    global_idx = generate_for_split("val", VAL_SAMPLES_PER_CLASS, BASE_SEED, global_idx,
                                    metadata_rows, generated_paths)

dt = time.time() - t0

# ----------------------------
# Sauvegarde CSV unique
# ----------------------------
csv_path = OUT_DIR / f"pixcell_metadata_{int(time.time())}.csv"
if len(metadata_rows) > 0:
    with open(csv_path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=list(metadata_rows[0].keys()))
        writer.writeheader()
        writer.writerows(metadata_rows)

print(f"\n🎉 Terminé : {len(generated_paths)} images générées (train + {'val' if VAL_ENABLE else '—'})")
print(f"📁 Structure miroir : {OUT_DIR}/<classe>/image.png")
print(f"🕒 Temps total : {dt:.1f}s")
print(f"🧾 CSV unique : {csv_path}")


📂 TRAIN dataset: NCT-CRC-HE-100K | VAL dataset: CRC-VAL-HE-7K
⚖️ 9 classes: ['ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']
📦 TRAIN: 500 / classe (≈ 4500 au total)
📦  VAL : 125 / classe (en plus du TRAIN)
✅ Seuils par classe chargés depuis : /workspace/configs/seuils_par_classe.json
⚖️ Échantillonnage équilibré activé (500 images / classe).
✅ Dataset train prêt : 4500 échantillons (500/classe)


Génération PixCell (train):   0%|          | 0/1125 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

...

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]


🎉 Terminé : 5625 images générées (train + val)
📁 Structure miroir : /workspace/outputs/07_diffusion_model/pixcell_out_histo/<classe>/image.png
🕒 Temps total : 5572.3s
🧾 CSV unique : /workspace/outputs/07_diffusion_model/pixcell_out_histo/pixcell_metadata_1763635740.csv


In [22]:
# Vidage RAM GPU CPU
import gc, torch
gc.collect(); torch.cuda.empty_cache()


# -------- SAUVEGARDE DES OBJETS (pipeline ou UNet/VAE) -------

In [23]:
# ==== Export local de UNI2-h ====
from huggingface_hub import snapshot_download
from pathlib import Path
import os, torch

# Utiliser CHECKPOINTS_DIR défini dans la cellule précédente
if "CHECKPOINTS_DIR" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    CHECKPOINTS_DIR = PROJECT_ROOT / "checkpoints" / "07_diffusion_model"
CKPT_ROOT = CHECKPOINTS_DIR
UNI2H_DIR = Path(os.getenv("UNI2H_DIR", str(CHECKPOINTS_DIR / "UNI2-h")))
UNI2H_DIR.mkdir(parents=True, exist_ok=True)

# 1) Snapshot complet du repo (recommandé si accès gated)
try:
    snapshot_download(
        repo_id="MahmoodLab/UNI2-h",
        local_dir=str(UNI2H_DIR),
        local_dir_use_symlinks=False,
        token=os.getenv("HF_TOKEN"),
        allow_patterns=["*.bin","*.safetensors","*.json","*config*","*txt","*md"]
    )
    print("✅ UNI2-h: snapshot HF sauvegardé ->", UNI2H_DIR)
except Exception as e:
    print("[warn] snapshot_download a échoué:", e, "\n→ on sauvegarde le state_dict timm")
    torch.save(model.state_dict(), UNI2H_DIR / "model_timm_state_dict.bin")
    with open(UNI2H_DIR / "README_LOCAL.txt","w") as f:
        f.write("State dict timm sauvegardé (fallback).\n")
    print("✅ UNI2-h: state_dict timm sauvegardé ->", UNI2H_DIR)


/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

✅ UNI2-h: snapshot HF sauvegardé -> /workspace/checkpoints/07_diffusion_model/UNI2-h


In [24]:
# Étape 1 — Détection robuste du pipeline PixCell déjà chargé
from diffusers import DiffusionPipeline

# 1) essaie les noms usuels
PIPE = None
for name in ["pipe256", "pipe", "pixcell", "pipeline", "pixcell256", "pix_pipe"]:
    if name in globals() and isinstance(globals()[name], DiffusionPipeline):
        PIPE = globals()[name]
        break

# 2) sinon, scanne tout l'espace global
if PIPE is None:
    for v in globals().values():
        if isinstance(v, DiffusionPipeline):
            PIPE = v
            break

if PIPE is None:
    raise RuntimeError(
        "Aucun objet DiffusionPipeline détecté. Exécute d'abord la cellule qui charge PixCell (pipe256 = DiffusionPipeline.from_pretrained(...))."
    )

UNET = getattr(PIPE, "unet", None)
VAE  = getattr(PIPE, "vae",  None)
print(f"✅ Pipeline détectée: {type(PIPE).__name__} | UNet: {UNET.__class__.__name__ if UNET else 'None'} | VAE: {VAE.__class__.__name__ if VAE else 'None'}")


✅ Pipeline détectée: PixCellPipeline | UNet: None | VAE: AutoencoderKL


In [25]:
# Étape 2 — Sauver le code de la custom pipeline en local
# 1) Chemins
from pathlib import Path
import os
# Utiliser CHECKPOINTS_DIR défini dans la cellule précédente
if "CHECKPOINTS_DIR" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    CHECKPOINTS_DIR = PROJECT_ROOT / "checkpoints" / "07_diffusion_model"
CKPT_ROOT = CHECKPOINTS_DIR
CUSTOM_PIPE_DIR = CKPT_ROOT / "custom_pipelines" / "PixCell-pipeline"
CUSTOM_PIPE_DIR.mkdir(parents=True, exist_ok=True)

# 2) Snapshot du repo de la pipeline custom
from huggingface_hub import snapshot_download

local_custom_dir = snapshot_download(
    repo_id="StonyBrook-CVLab/PixCell-pipeline",  # le même que tu passes en custom_pipeline
    local_dir=str(CUSTOM_PIPE_DIR),
    local_dir_use_symlinks=False,
    token=os.getenv("HF_TOKEN", None),
    # on garde tout (py, config, etc.)
)
print("✅ Custom pipeline sauvegardée ici :", local_custom_dir)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Custom pipeline sauvegardée ici : /workspace/checkpoints/07_diffusion_model/custom_pipelines/PixCell-pipeline


In [26]:
# Exporter PixCell au format Diffusers (model_index.json + unet/ + vae/)
from diffusers import DiffusionPipeline

# Utiliser CHECKPOINTS_DIR défini dans la cellule précédente
if "CHECKPOINTS_DIR" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    CHECKPOINTS_DIR = PROJECT_ROOT / "checkpoints" / "07_diffusion_model"
if "CKPT_ROOT" not in globals():
    CKPT_ROOT = CHECKPOINTS_DIR
PIXCELL_DIR = CKPT_ROOT / "pixcell256-gated"
PIXCELL_DIR.mkdir(parents=True, exist_ok=True)

# Détection de ta pipeline déjà chargée (pipe256)
PIPE = None
try:
    PIPE = pipe256  # ton objet existant
except NameError:
    # (fallback ultra-sûr : scanne les globals)
    for v in globals().values():
        if isinstance(v, DiffusionPipeline):
            PIPE = v
            break
    assert PIPE is not None, "Recharge d'abord la pipeline PixCell dans ce notebook."

PIPE.save_pretrained(PIXCELL_DIR, safe_serialization=True)
print("✅ Poids PixCell exportés ->", PIXCELL_DIR)


✅ Poids PixCell exportés -> /workspace/checkpoints/07_diffusion_model/pixcell256-gated


In [27]:
from pathlib import Path
import torch
from diffusers import DiffusionPipeline

# Utiliser CHECKPOINTS_DIR défini dans la cellule précédente
if "CHECKPOINTS_DIR" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    CHECKPOINTS_DIR = PROJECT_ROOT / "checkpoints" / "07_diffusion_model"
CKPT_ROOT = CHECKPOINTS_DIR
PIXCELL_DIR = CKPT_ROOT / "pixcell256-gated"
CUSTOM_PIPE_DIR = CKPT_ROOT / "custom_pipelines" / "PixCell-pipeline"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.float16 if device == "cuda" else torch.float32

pipe_chk = DiffusionPipeline.from_pretrained(
    PIXCELL_DIR,
    custom_pipeline=str(CUSTOM_PIPE_DIR),  # ← on donne le chemin des .py
    trust_remote_code=True,
    torch_dtype=dtype,
).to(device)

print("✅ Reload OK:", type(pipe_chk).__name__)


Keyword arguments {'trust_remote_code': True} are not expected by PixCellPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

The config attributes {'double_self_attention': False, 'num_vector_embeds': None, 'only_cross_attention': False, 'use_linear_projection': False} were passed to PixCellTransformer2DModel, but are not expected and will be ignored. Please verify your config.json configuration file.
The config attributes {'flow_shift': 1.0, 'use_flow_sigmas': False} were passed to DPMSolverMultistepScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


✅ Reload OK: PixCellPipeline


## 💾 Création du modèle de référence pour re-load rapide


In [28]:
# %% 💾 Création du modèle de référence .pt pour re-load rapide
import torch
from pathlib import Path
import os
from datetime import datetime

# Vérifications préalables
assert "pipe256" in globals(), "⚠️ pipe256 doit être chargée. Exécutez d'abord les cellules de chargement."
assert "uni" in globals(), "⚠️ UNI2-h (uni) doit être chargé. Exécutez d'abord les cellules de chargement."
assert "tfm" in globals(), "⚠️ Transform (tfm) doit être défini. Exécutez d'abord les cellules de chargement."

# Utiliser MODELS_DIR et CHECKPOINTS_DIR définis dans la cellule précédente
if "MODELS_DIR" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    MODELS_DIR = PROJECT_ROOT / "models"
if "CHECKPOINTS_DIR" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    CHECKPOINTS_DIR = PROJECT_ROOT / "checkpoints" / "07_diffusion_model"

MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Code du monkey-patch (à appliquer après rechargement)
monkey_patch_code = '''
def _patched_forward(self, *f_args, **f_kwargs):
    """
    Compatibilité maximale avec tous les schémas d'appel:
    forward(self, timestep, resolution, aspect_ratio, [batch_size], [hidden_dtype])
    + variantes avec kwargs.
    Injecte resolution/aspect_ratio si absents.
    """
    import torch
    # --- 1) Extraire les params quelle que soit la forme d'appel
    timestep = f_args[0] if len(f_args) > 0 else f_kwargs.get("timestep", None)
    
    resolution   = f_kwargs.get("resolution",   None)
    aspect_ratio = f_kwargs.get("aspect_ratio", None)
    if resolution is None and len(f_args) > 1:
        resolution = f_args[1]
    if aspect_ratio is None and len(f_args) > 2:
        aspect_ratio = f_args[2]
    
    batch_size   = f_kwargs.get("batch_size",   None)
    hidden_dtype = f_kwargs.get("hidden_dtype", None)
    if batch_size is None and len(f_args) > 3:
        batch_size = f_args[3]
    if hidden_dtype is None and len(f_args) > 4:
        hidden_dtype = f_args[4]
    
    # --- 2) Déterminer B et device
    if batch_size is not None:
        if isinstance(batch_size, torch.Tensor):
            B = int(batch_size.item())
        else:
            try:
                B = int(batch_size)
            except Exception:
                B = 1
    else:
        try:
            B = int(getattr(timestep, "shape", [1])[0])
        except Exception:
            B = 1
    
    if isinstance(timestep, torch.Tensor):
        device = timestep.device
    else:
        try:
            device = next(self.parameters()).device
        except Exception:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # --- 3) Injecter les valeurs manquantes
    if resolution is None:
        resolution = torch.full((B,), 256.0, device=device, dtype=torch.float32)
    if aspect_ratio is None:
        aspect_ratio = torch.full((B,),  1.0, device=device, dtype=torch.float32)
    
    # --- 4) Appeler l'original
    return orig_forward(
        timestep,
        resolution,
        aspect_ratio,
        batch_size=batch_size,
        hidden_dtype=hidden_dtype,
    )
'''

# Structure du modèle de référence
reference_model = {
    "version": "1.0",
    "model_type": "pixcell256_inference",
    "description": "Modèle de référence pour inférence PixCell avec UNI2-h et monkey-patch",
    "created_at": datetime.now().isoformat(),
    
    # PixCell Pipeline
    "pixcell": {
        "pixcell_dir": str(CHECKPOINTS_DIR / "pixcell256-gated"),
        "custom_pipeline_dir": str(CHECKPOINTS_DIR / "custom_pipelines" / "PixCell-pipeline"),
        "load_config": {
            "device": "cuda" if torch.cuda.is_available() else "cpu",
            "dtype": "float16" if torch.cuda.is_available() else "float32",
            "trust_remote_code": True,
        },
    },
    
    # UNI2-h (OBLIGATOIRE pour inférence histopatho)
    "uni2h": {
        "repo_id": "hf-hub:MahmoodLab/UNI2-h",
        "local_dir": str(CHECKPOINTS_DIR / "UNI2-h"),
        "requires_token": True,
        "timm_kwargs": {
            "img_size": 224,
            "patch_size": 14,
            "depth": 24,
            "num_heads": 24,
            "embed_dim": 1536,
            "mlp_ratio": 2.66667 * 2,
            "init_values": 1e-5,
            "num_classes": 0,
            "no_embed_class": True,
            "reg_tokens": 8,
            "dynamic_img_size": True,
            "mlp_layer": "timm.layers.SwiGLUPacked",
            "act_layer": "torch.nn.SiLU",
        },
        "device": "cuda" if torch.cuda.is_available() else "cpu",
    },
    
    # Monkey-patch (OBLIGATOIRE)
    "monkey_patch": {
        "target": "pipe256.transformer.adaln_single.emb.forward",
        "description": "Patch pour PixArtAlphaCombinedTimestepSizeEmbeddings.forward",
        "code": monkey_patch_code.strip(),
        "required_imports": ["types", "torch"],
    },
    
    # Paramètres d'inférence par défaut
    "default_inference": {
        "guidance_scale": 2.0,
        "steps": 28,
        "height": 256,
        "width": 256,
    },
    
    # Optimisations VRAM
    "optimizations": {
        "attention_slicing": True,
        "vae_slicing": True,
        "vae_tiling": False,
    },
}

# Sauvegarder le modèle de référence
model_ref_path = MODELS_DIR / "pixcell256_reference.pt"
torch.save(reference_model, model_ref_path, _use_new_zipfile_serialization=False)

print("✅ Modèle de référence créé:")
print(f"   📁 Chemin: {model_ref_path}")
print(f"   📊 Version: {reference_model['version']}")
print(f"   📅 Créé le: {reference_model['created_at']}")
print(f"\n💡 Contenu:")
print(f"   - PixCell Pipeline: {reference_model['pixcell']['pixcell_dir']}")
print(f"   - UNI2-h: {reference_model['uni2h']['repo_id']}")
print(f"   - Monkey-patch: {reference_model['monkey_patch']['target']}")
print(f"   - Paramètres par défaut: guidance={reference_model['default_inference']['guidance_scale']}, steps={reference_model['default_inference']['steps']}")


✅ Modèle de référence créé:
   📁 Chemin: /workspace/models/pixcell256_reference.pt
   📊 Version: 1.0
   📅 Créé le: 2025-11-20T11:49:03.782183

💡 Contenu:
   - PixCell Pipeline: /workspace/checkpoints/07_diffusion_model/pixcell256-gated
   - UNI2-h: hf-hub:MahmoodLab/UNI2-h
   - Monkey-patch: pipe256.transformer.adaln_single.emb.forward
   - Paramètres par défaut: guidance=2.0, steps=28


## 🔄 Test de rechargement rapide avec le modèle de référence


In [17]:
# === Toggles mémoire : checkpointing, attn slicing/xformers, VAE tiling ===
try:
    BACKBONE.enable_gradient_checkpointing()
    print("✓ Gradient checkpointing activé")
except Exception as e:
    print("↷ Checkpointing indisponible sur ce backbone :", type(e).__name__)

# xFormers / FlashAttention / slicing
ok_attn = False
try:
    BACKBONE.enable_xformers_memory_efficient_attention()
    print("✓ xFormers attention"); ok_attn = True
except Exception:
    try:
        from diffusers.models.attention_processor import AttnProcessor2_0
        BACKBONE.set_attn_processor(AttnProcessor2_0())
        print("✓ AttnProcessor2_0"); ok_attn = True
    except Exception:
        try:
            BACKBONE.enable_attention_slicing("max")
            print("✓ Attention slicing (fallback)"); ok_attn = True
        except Exception as e:
            print("↷ Pas de backend attn optimisé :", type(e).__name__)

# VAE encode/decode par tuiles (gros gain mémoire)
try:
    VAE.enable_tiling(); VAE.enable_slicing()
    print("✓ VAE tiling + slicing")
except Exception as e:
    print("↷ VAE tiling/slicing non supporté :", type(e).__name__)

# Option : réduis la résolution si tu es short
IMG_SIZE = int(globals().get('IMG_SIZE', 256))
# IMG_SIZE = 192  # ← décommente si besoin d’un cran de marge


↷ Checkpointing indisponible sur ce backbone : NameError
↷ Pas de backend attn optimisé : NameError
↷ VAE tiling/slicing non supporté : NameError


In [18]:
# %% 🧹 OOM cleanup — ferme figures, libère tenseurs, retire hooks, vide caches CUDA
import gc, sys
import torch
import matplotlib
import types

def _mb(x): 
    return f"{x/1024/1024:.1f} MB"

# 1) Fermer toutes les figures Matplotlib
try:
    import matplotlib.pyplot as plt
    plt.close('all')
    print("✔️ Figures Matplotlib fermées.")
except Exception as e:
    print("ℹ️ plt.close('all') ignoré:", e)

# 2) Retirer le hook 'noise-only' si présent
try:
    _pipe = globals().get("pipe", None) or globals().get("pipe256", None)
    if _pipe is not None and getattr(_pipe.transformer, "_noise_only_handle", None) is not None:
        _pipe.transformer._noise_only_handle.remove()
        _pipe.transformer._noise_only_handle = None
        print("✔️ Hook 'noise-only' retiré.")
except Exception as e:
    print("ℹ️ Retrait hook ignoré:", e)

# 3) Supprimer variables temporaires fréquentes
to_del = [
    "out","images","pils","pils_show","x01","latents","noise","noisy","pred","target",
    "pos_uni","neg_uni","pos_enc","neg_enc","g","batch","dl_iter"
]
freed = 0
for name in to_del:
    if name in globals():
        try:
            obj = globals()[name]
            del globals()[name]
            freed += 1
        except Exception:
            pass
print(f"✔️ Variables temporaires supprimées: {freed}")

# 4) Forcer le GC Python
collected = gc.collect()
print(f"✔️ GC collecté: {collected} objets")

# 5) Vidage des caches CUDA
if torch.cuda.is_available():
    try:
        torch.cuda.synchronize()
    except Exception:
        pass
    try:
        before_alloc = torch.cuda.memory_allocated()
        before_resrv = torch.cuda.memory_reserved()
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        torch.cuda.reset_peak_memory_stats()
        after_alloc  = torch.cuda.memory_allocated()
        after_resrv  = torch.cuda.memory_reserved()
        print(f"✔️ CUDA cache vidé. Alloc {_mb(before_alloc)} → {_mb(after_alloc)} | Reserved {_mb(before_resrv)} → {_mb(after_resrv)}")
    except Exception as e:
        print("ℹ️ Cleanup CUDA ignoré:", e)
else:
    print("ℹ️ CUDA non disponible.")

print("✅ Ménage fini — si besoin, relance la génération avec un batch/steps plus petits (ex: N_SHOW=2, GEN_STEPS=12).")


✔️ Figures Matplotlib fermées.
✔️ Variables temporaires supprimées: 0
✔️ GC collecté: 24 objets
✔️ CUDA cache vidé. Alloc 3963.1 MB → 3963.1 MB | Reserved 5552.0 MB → 4156.0 MB
✅ Ménage fini — si besoin, relance la génération avec un batch/steps plus petits (ex: N_SHOW=2, GEN_STEPS=12).


In [20]:
# %% 🔄 Test de rechargement rapide avec le modèle de référence
import torch
from pathlib import Path
import os
import types
from diffusers import DiffusionPipeline
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from PIL import Image

# Utiliser MODELS_DIR défini dans la cellule précédente
if "MODELS_DIR" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    MODELS_DIR = PROJECT_ROOT / "models"

# Charger le modèle de référence
model_ref_path = MODELS_DIR / "pixcell256_reference.pt"
assert model_ref_path.exists(), f"⚠️ Modèle de référence introuvable: {model_ref_path}"

print("📂 Chargement du modèle de référence...")
ref = torch.load(model_ref_path, map_location="cpu", weights_only=False)

print(f"✅ Modèle de référence chargé:")
print(f"   Version: {ref['version']}")
print(f"   Type: {ref['model_type']}")
print(f"   Créé le: {ref.get('created_at', 'N/A')}")

# 1) Recharger UNI2-h
print("\n🔄 Rechargement de UNI2-h...")
device = ref["uni2h"]["device"]
timm_kwargs = ref["uni2h"]["timm_kwargs"].copy()

# Convertir les strings en types réels
if timm_kwargs["mlp_layer"] == "timm.layers.SwiGLUPacked":
    timm_kwargs["mlp_layer"] = timm.layers.SwiGLUPacked
if timm_kwargs["act_layer"] == "torch.nn.SiLU":
    timm_kwargs["act_layer"] = torch.nn.SiLU

uni_reloaded = timm.create_model(
    ref["uni2h"]["repo_id"],
    pretrained=True,
    **timm_kwargs
).eval().to(device)

# Créer le transform
cfg = resolve_data_config(uni_reloaded.pretrained_cfg, model=uni_reloaded)
tfm_reloaded = create_transform(**cfg)

print(f"✅ UNI2-h rechargé: {next(uni_reloaded.parameters()).dtype} | device: {device}")

# 2) Recharger PixCell Pipeline
print("\n🔄 Rechargement de PixCell Pipeline...")
pixcell_dir = Path(ref["pixcell"]["pixcell_dir"])
custom_pipe_dir = Path(ref["pixcell"]["custom_pipeline_dir"])

# Résoudre les chemins relatifs si nécessaire
if not pixcell_dir.is_absolute():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    pixcell_dir = PROJECT_ROOT / pixcell_dir
    custom_pipe_dir = PROJECT_ROOT / custom_pipe_dir

load_config = ref["pixcell"]["load_config"]
dtype_map = {"float16": torch.float16, "float32": torch.float32}
torch_dtype = dtype_map[load_config["dtype"]]

pipe_reloaded = DiffusionPipeline.from_pretrained(
    str(pixcell_dir),
    custom_pipeline=str(custom_pipe_dir),
    trust_remote_code=load_config["trust_remote_code"],
    torch_dtype=torch_dtype,
).to(load_config["device"])

# Appliquer les optimisations VRAM
opt = ref["optimizations"]
if opt["attention_slicing"]:
    pipe_reloaded.enable_attention_slicing()
if opt["vae_slicing"]:
    pipe_reloaded.vae.enable_slicing()
if opt.get("vae_tiling", False):
    pipe_reloaded.vae.enable_tiling()

print(f"✅ PixCell Pipeline rechargée: {type(pipe_reloaded).__name__}")

# 3) Appliquer le monkey-patch
print("\n🔄 Application du monkey-patch...")
adaln = pipe_reloaded.transformer.adaln_single
emb_mod = adaln.emb
orig_forward = emb_mod.forward

# Exécuter le code du patch
patch_code = ref["monkey_patch"]["code"]
# Créer un contexte avec orig_forward disponible
exec_globals = {"torch": torch, "orig_forward": orig_forward, "getattr": getattr}
exec(patch_code, exec_globals)
_patched_forward = exec_globals["_patched_forward"]

# Appliquer le patch
emb_mod.forward = types.MethodType(_patched_forward, emb_mod)
print(f"✅ Monkey-patch appliqué: {ref['monkey_patch']['target']}")

# 4) Test de génération rapide (avec nettoyage mémoire avant)
print("\n🧪 Test de génération...")

# Nettoyage mémoire avant le test pour éviter OOM
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

test_img = Image.new("RGB", (224, 224), color=(128, 128, 128))

# Embedding UNI
x = tfm_reloaded(test_img.convert("RGB")).unsqueeze(0).to(device)
with torch.autocast(device_type="cuda", enabled=(device=="cuda")):
    e = uni_reloaded(x)
emb_test = e.unsqueeze(1).to(device, dtype=torch_dtype)

# Génération
B = emb_test.shape[0]
uncond_test = pipe_reloaded.get_unconditional_embedding(B).to(device=emb_test.device, dtype=emb_test.dtype)

default_params = ref["default_inference"]
# Utiliser moins de steps pour le test (évite OOM)
test_steps = min(5, default_params["steps"])  # Max 15 steps pour le test
generator = torch.Generator(device=device).manual_seed(42)

with torch.autocast(device_type="cuda", enabled=(device=="cuda")):
    result = pipe_reloaded(
        uni_embeds=emb_test,
        negative_uni_embeds=uncond_test,
        guidance_scale=default_params["guidance_scale"],
        num_inference_steps=test_steps,  # Utiliser test_steps au lieu de default_params["steps"]
        num_images_per_prompt=1,
        generator=generator,
        height=default_params["height"],
        width=default_params["width"],
        output_type="pil",
    )

gen_img = result.images[0] if hasattr(result, "images") else result[0]
print(f"✅ Génération réussie: {gen_img.size}")

# Sauvegarder l'image de test
if "SAMPLES_DIR" not in globals():
    if "PROJECT_ROOT" not in globals():
        PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "..")).resolve()
    SAMPLES_DIR = PROJECT_ROOT / "samples" / "07_diffusion_model"
SAMPLES_DIR.mkdir(parents=True, exist_ok=True)
test_path = SAMPLES_DIR / "reload_test.png"
gen_img.save(test_path)
print(f"💾 Image de test sauvegardée: {test_path}")

print("\n✅ Rechargement complet réussi !")
print(f"   - UNI2-h: ✅")
print(f"   - PixCell Pipeline: ✅")
print(f"   - Monkey-patch: ✅")
print(f"   - Génération de test: ✅")


📂 Chargement du modèle de référence...
✅ Modèle de référence chargé:
   Version: 1.0
   Type: pixcell256_inference
   Créé le: 2025-11-20T11:49:03.782183

🔄 Rechargement de UNI2-h...


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 7.63 GiB of which 22.56 MiB is free. Including non-PyTorch memory, this process has 7.49 GiB memory in use. Of the allocated memory 7.07 GiB is allocated by PyTorch, and 304.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [21]:
import inspect
from diffusers.models import UNet2DConditionModel, AutoencoderKL

# 2a) liste des composants déclarés
comp = getattr(pipe_chk, "components", None)
print("Composants:", list(comp.keys()) if comp else "—")

# 2b) chercher UNet / VAE par type, même si le nom n’est pas 'unet'
def find_module_of_type(pipe, cls):
    if hasattr(pipe, "components") and isinstance(pipe.components, dict):
        for k, v in pipe.components.items():
            if isinstance(v, cls):
                return k, v
    # fallback: scan des attributs
    for name in dir(pipe):
        if name.startswith("_"): 
            continue
        try:
            val = getattr(pipe, name)
        except Exception:
            continue
        if isinstance(val, cls):
            return name, val
    return None, None

u_name, u_mod = find_module_of_type(pipe_chk, UNet2DConditionModel)
v_name, v_mod = find_module_of_type(pipe_chk, AutoencoderKL)
print("UNet :", u_name, type(u_mod) if u_mod else None)
print("VAE  :", v_name, type(v_mod) if v_mod else None)

# 2c) si on a trouvé un UNet, récupère cross_attention_dim ; sinon on passera en "détection par essai"
CROSS_DIM = getattr(getattr(u_mod, "config", None), "cross_attention_dim", None)
print("cross_attention_dim (si disponible) ->", CROSS_DIM)

# 2d) Vérifier la signature d'appel: quel nom de param pour le conditionnement ?
print("Signature __call__:", inspect.signature(pipe_chk.__call__))


NameError: name 'pipe_chk' is not defined